In [9]:
# Import modules
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import itertools as it
import statsmodels.api as sm
from statsmodels.tools.eval_measures import rmse
from itertools import combinations
from statsmodels.stats.outliers_influence import variance_inflation_factor
import csv
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

In [10]:
# Import the raw UAV data from a CSV file
pathToFile = 'F:/Redaction/Bernadouze/Submitted/R2/Data_repository/Dataset_with_indices.csv'
rawData = pd.read_csv(pathToFile,sep=',')
UAVData = pd.DataFrame(rawData)
UAVData.head(40)

,Plot,Season,GEP (gCO2.m².hour-1),ER (gCO2.m².hour-1),NEE (gCO2.m².hour-1),B,G,R,RE,NIR,...,GLI,GNDVI,LCI,NDRE,NDVI,OSAVI,RCC,RVI,SIPI,TVI
0,P1,May,-1.86,1.30,-0.56,0.04,0.09,0.10,0.30,0.43,...,0.125000,0.653846,0.245283,0.178082,0.622642,0.478261,0.434783,4.300000,1.181818,1.059548
1,P3,May,-1.47,1.28,-0.19,0.04,0.09,0.07,0.25,0.40,...,0.241379,0.632653,0.319149,0.230769,0.702128,0.523810,0.350000,5.714286,1.090909,1.096416
2,P4,May,-1.25,0.89,-0.36,0.04,0.10,0.08,0.28,0.42,...,0.250000,0.615385,0.280000,0.200000,0.680000,0.515152,0.363636,5.250000,1.117647,1.086278
3,P5,May,-1.84,1.19,-0.65,0.04,0.07,0.05,0.18,0.34,...,0.217391,0.658537,0.410256,0.307692,0.743590,0.527273,0.312500,6.800000,1.034483,1.115164
4,P6,May,-1.27,0.96,-0.31,0.05,0.10,0.09,0.24,0.37,...,0.176471,0.574468,0.282609,0.213115,0.608696,0.451613,0.375000,4.111111,1.142857,1.052946
5,P7,May,-1.20,0.86,-0.34,0.03,0.06,0.04,0.18,0.32,...,0.263158,0.684211,0.388889,0.280000,0.777778,0.538462,0.307692,8.000000,1.035714,1.130388
6,P8,May,-0.93,0.97,0.04,0.04,0.06,0.07,0.18,0.26,...,0.043478,0.625000,0.242424,0.181818,0.575758,0.387755,0.411765,3.714286,1.157895,1.037187
7,P9,May,-1.12,0.94,-0.18,0.03,0.09,0.07,0.25,0.36,...,0.285714,0.600000,0.255814,0.180328,0.674419,0.491525,0.368421,5.142857,1.137931,1.083706
8,P1,June,-1.46,0.80,-0.66,0.02,0.06,0.05,0.22,0.35,...,0.263158,0.707317,0.325000,0.228070,0.750000,0.535714,0.384615,7.000000,1.100000,1.118034
9,P3,June,-1.23,0.76,-0.47,0.02,0.06,0.04,0.19,0.34,...,0.333333,0.700000,0.394737,0.283019,0.789474,0.555556,0.333333,8.500000,1.066667,1.135550


In [7]:
# Resolve colinearity issue by eliminating colinear explanatory variables

UAVX = UAVData[{'PAR','TEMP','DOY', # meteo
             'B','G','R','RE','NIR', # bands
             'BCC','BRI','BWDRVI','CIgreen','CVI','EVI','EVI2','GARI','GCC','GLI','GNDVI','LCI','NDRE','NDVI','OSAVI','RCC','RVI','SIPI','TVI'}] # VIs

np.seterr(divide='ignore', invalid='ignore')
def calculate_vif_(X, thresh=5):
    X = X.assign(const=1)  # faster than add_constant from statsmodels
    variables = list(range(X.shape[1]))
    dropped = True
    while dropped:
        dropped = False
        vif = [variance_inflation_factor(X.iloc[:, variables].values, ix)
               for ix in range(X.iloc[:, variables].shape[1])]
        vif = vif[:-1]  # don't let the constant be removed in the loop.
        maxloc = vif.index(max(vif))
        if max(vif) > thresh:
            print('dropping \'' + X.iloc[:, variables].columns[maxloc] +'\' at index: ' + str(maxloc) + ' with VIF =' + str(max(vif)))
            del variables[maxloc]
            dropped = True
            print('Remaining variables:', X.columns[variables[:-1]])
    return 

calculate_vif_(UAVX)

dropping 'BCC' at index: 3 with VIF =inf
Remaining variables: Index(['G', 'NDRE', 'BRI', 'EVI2', 'RCC', 'NIR', 'B', 'BWDRVI', 'SIPI', 'NDVI',
       'GNDVI', 'EVI', 'PAR', 'TEMP', 'GLI', 'DOY', 'LCI', 'GCC', 'GARI', 'R',
       'CVI', 'RE', 'RVI', 'CIgreen', 'TVI', 'OSAVI'],
      dtype='object')
dropping 'NDVI' at index: 9 with VIF =10149842.258906439
Remaining variables: Index(['G', 'NDRE', 'BRI', 'EVI2', 'RCC', 'NIR', 'B', 'BWDRVI', 'SIPI',
       'GNDVI', 'EVI', 'PAR', 'TEMP', 'GLI', 'DOY', 'LCI', 'GCC', 'GARI', 'R',
       'CVI', 'RE', 'RVI', 'CIgreen', 'TVI', 'OSAVI'],
      dtype='object')
dropping 'EVI2' at index: 3 with VIF =830898.4406166737
Remaining variables: Index(['G', 'NDRE', 'BRI', 'RCC', 'NIR', 'B', 'BWDRVI', 'SIPI', 'GNDVI', 'EVI',
       'PAR', 'TEMP', 'GLI', 'DOY', 'LCI', 'GCC', 'GARI', 'R', 'CVI', 'RE',
       'RVI', 'CIgreen', 'TVI', 'OSAVI'],
      dtype='object')
dropping 'GLI' at index: 12 with VIF =315067.1978694766
Remaining variables: Index(['G', 'NDRE', 'B

In [4]:
# Test models: each variable combinaison over 100 train/test splits
results_UAV = pd.DataFrame(columns=['VarToPred', 'SplitId', 'NbOfExpVar', 'ExpVar','R','RMSE','AIC', 'Model']) # Put results in a dataframe
c = 0

predVar = ['NEE (gCO2.m².hour-1)', 'ER (gCO2.m².hour-1)', 'GEP (gCO2.m².hour-1)']

for p in predVar: # Loop by variable to predict
    y = UAVData[p]
    X = UAVData[{'NDRE', 'BRI', 'NIR', 'BWDRVI', 'PAR', 'TEMP', 'CVI'}]
    for i in range(1,len(X.columns)+1): # Loop by the number of explanatory variables (X)
        SelectedFeatureNames = list(combinations(X.columns,i)) # List all combinations
        for elem in SelectedFeatureNames:
            for SplitNb in range(100): # Split the dataset in train and test dataset
                c = c+1
                train, test = train_test_split(UAVData, test_size=0.4)
                X_train = train.loc[:,elem]
                y_train = train.loc[:,p]
                X_test = test.loc[:,elem]
                y_test = test.loc[:,p]
                
                # Fit a model with train dataset
                model = sm.OLS(y_train,sm.add_constant(X_train,has_constant='add')).fit()
                
                # Make prediction on the test dataset
                ypred = model.predict(sm.add_constant(X_test,has_constant='add'))
                
                # Performance 
                AIC = model.aic
                R = np.corrcoef(y_test, ypred)
                RMSE = rmse(y_test, ypred)
                
                # Write it in the dataframe
                results_UAV.loc[c, 'VarToPred'] = p
                results_UAV.loc[c, 'SplitId'] = SplitNb
                results_UAV.loc[c, 'NbOfExpVar'] = i
                results_UAV.at[c, 'ExpVar'] = elem
                results_UAV.loc[c, 'R'] = R[0,1]
                results_UAV.loc[c, 'RMSE'] = RMSE
                results_UAV.loc[c, 'AIC'] = AIC
                results_UAV.loc[c, 'Model'] = model
                
# Save to dbf
results_UAV.to_csv('F:/Redaction/Bernadouze/Submitted/R2/Data_repository/ModelsUAV.csv',index=False)
results_UAV

,VarToPred,SplitId,NbOfExpVar,ExpVar,R,RMSE,AIC,Model
1,NEE (gCO2.m².hour-1),0,1,"(NDRE,)",0.447557,0.307877,12.608078,<statsmodels.regression.linear_model.Regressio...
2,NEE (gCO2.m².hour-1),1,1,"(NDRE,)",0.337475,0.34151,8.850331,<statsmodels.regression.linear_model.Regressio...
3,NEE (gCO2.m².hour-1),2,1,"(NDRE,)",0.682306,0.295529,15.038961,<statsmodels.regression.linear_model.Regressio...
4,NEE (gCO2.m².hour-1),3,1,"(NDRE,)",0.385386,0.374839,6.068473,<statsmodels.regression.linear_model.Regressio...
5,NEE (gCO2.m².hour-1),4,1,"(NDRE,)",0.29771,0.295229,14.00352,<statsmodels.regression.linear_model.Regressio...
...,...,...,...,...,...,...,...,...
38096,GEP (gCO2.m².hour-1),95,7,"(NDRE, BRI, TEMP, BWDRVI, CVI, PAR, NIR)",-0.153294,3.95351,31.676199,<statsmodels.regression.linear_model.Regressio...
38097,GEP (gCO2.m².hour-1),96,7,"(NDRE, BRI, TEMP, BWDRVI, CVI, PAR, NIR)",0.616186,0.559921,23.612425,<statsmodels.regression.linear_model.Regressio...
38098,GEP (gCO2.m².hour-1),97,7,"(NDRE, BRI, TEMP, BWDRVI, CVI, PAR, NIR)",0.612478,0.504384,36.724398,<statsmodels.regression.linear_model.Regressio...
38099,GEP (gCO2.m².hour-1),98,7,"(NDRE, BRI, TEMP, BWDRVI, CVI, PAR, NIR)",0.567717,0.569381,26.423526,<statsmodels.regression.linear_model.Regressio...
